In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [ ]:
!pip install hazm

     |████████████████████████████████| 317kB 6.8MB/s 
     |████████████████████████████████| 1.4MB 11.9MB/s 
     |████████████████████████████████| 235kB 28.4MB/s 
  Created wheel for nltk: filename=nltk-3.3-cp37-none-any.whl size=1394488 sha256=59f0086b551d0aebf0f75ff08796362d80eba2e013321e7baccd2c16879d05b1
  Stored in directory: /root/.cache/pip/wheels/d1/ab/40/3bceea46922767e42986aef7606a600538ca80de6062dc266c
  Created wheel for libwapiti: filename=libwapiti-0.2.1-cp37-cp37m-linux_x86_64.whl size=154162 sha256=0374229b39b4e1bf8722379273ce0f63459ec7bb7aec261985b8fc8f9d5b5da5
  Stored in directory: /root/.cache/pip/wheels/66/15/54/4510dce8bb958b1cdd2c47425cbd1e1eecc0480ac9bb1fb9ab
Successfully built nltk libwapiti
  Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow.keras import optimizers
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Input, Embedding, Dropout, GlobalMaxPool1D,MaxPool1D 
from tensorflow.compat.v1.keras.layers import CuDNNGRU

from tensorflow.keras.layers import LSTM, Bidirectional
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.optimizers import Adam,RMSprop
# Preprocessing
from hazm import *

In [ ]:
train = pd.read_csv("/content/drive/MyDrive/Deep/NLP/persian_news/fixed_train.csv", sep='	')
test = pd.read_csv("/content/drive/MyDrive/Deep/NLP/persian_news/fixed_test.csv", sep='	')
dev = pd.read_csv("/content/drive/MyDrive/Deep/NLP/persian_news/fixed_dev.csv", sep='	')

In [ ]:
train=train.append(dev)
train

,Unnamed: 0,content,label,label_id
0,0,گزارش خبرنگار حوزه بهداشت درمان گروه علمی پزشک...,پزشکی,7
1,1,گزارش خبرنگار فوتبال فوتسال گروه ورزشی باشگاه ...,ورزشی,6
2,2,بهروز اکرمی، گفتگو خبرنگار اجتماعی باشگاه خبرن...,اجتماعی,0
3,3,گزارش خبرنگار حوزه شهری گروه اجتماعی باشگاه خب...,اجتماعی,0
4,4,گزارش باشگاه خبرنگاران نقل روابط عمومی تماشاخا...,فرهنگی هنری,5
...,...,...,...,...
1475,1475,گزارش خبرنگارکلینیک باشگاه خبرنگاران؛ خونی، حو...,پزشکی,7
1476,1476,رضا قدیمی رئیس کمیته منابع انسانی ستاد اربعین ...,اجتماعی,0
1477,1477,گزارش خبرنگار گروه علمی دانشگاهی خبرگزاری فارس...,علمی فناوری,4
1478,1478,گزارش خبرنگار حوزه کلینیک گروه علمی پزشکی باشگ...,پزشکی,7


In [ ]:
sentence_train = train['content']
label_train = train['label_id']

print('Number of training sentence: ', sentence_train.shape)
print('Number of training label: ', label_train.shape)

Number of training sentence:  (14794,)
Number of training label:  (14794,)


In [ ]:
from collections import Counter
cnt = Counter(label_train)
cnt = dict(cnt)
print(cnt)

{7: 1876, 6: 1243, 0: 1953, 5: 2302, 4: 2192, 3: 2042, 2: 1778, 1: 1408}


In [ ]:
# Convert dataframes to numpy arrays
sentence_train = np.asarray(sentence_train)
label_train = np.asarray(label_train)

In [ ]:
# Train Prepare labels for categorical prediction
categorical_label_train = to_categorical(label_train, 8)
categorical_label_train

array([[0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 1., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 1., 0., ..., 0., 0., 0.]], dtype=float32)

In [ ]:
import re
normalizer = Normalizer()
lemmatizer = Lemmatizer()

# turn a doc into clean tokens
def clean_data(doc):
    doc = normalizer.normalize(doc) # Normalize document using Hazm Normalizer
    tokenized = word_tokenize(doc)  # Tokenize text
    
    tokens = []
    for token in tokenized:
      token = re.sub("[،:.,;()/+]", " ", token) 
      token = re.sub(r"\!+","!", token)
      token = re.sub(r"\؟+","؟", token)
      token = re.sub(r"\u200c", " ", token)
      tokens.append(token)

    tokens = [w for w in tokens if not len(w) <= 1] # single character removal 
    tokens = [w for w in tokens if not w.isdigit()] # digit remove
    tokens = [lemmatizer.lemmatize(w) for w in tokens] # Lemmatize sentence words using Hazm Lemmatizer
    
    tokens = ' '.join(tokens)
    return tokens

In [ ]:
# Apply preprocessing step to training data
train_docs = np.empty_like(sentence_train)
for index, document in enumerate(sentence_train):
  train_docs[index] = clean_data(document)

In [ ]:
num_words = 2000

# Create the tokenizer
tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(train_docs)
encoded_docs = tokenizer.texts_to_sequences(train_docs)

In [ ]:
# Find maximum length of training sentences
max_length = max([len(s.split()) for s in train_docs])
print(max_length)

# Pad embeded training sequences
x_train_padded = pad_sequences(encoded_docs, maxlen=max_length, padding='post')

10173


In [ ]:
#vocabulary size
vocab_size = len(tokenizer.word_index)
vocab_size

72348

In [ ]:
model = Sequential()

model.add(Embedding(vocab_size, 185, input_length=max_length))
model.add(CuDNNGRU(185))

model.add(GlobalMaxPool1D())
model.add(Dropout(0.25))
model.add(Dense(300, activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(8, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer=RMSprop(), metrics=['accuracy'])

model.summary()
batch_size = 64
epochs = 10

hist = model.fit(x_train_padded, categorical_label_train, validation_split=0.2, batch_size=batch_size, epochs=epochs)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 10173, 185)        13384380  
_________________________________________________________________
cu_dnngru (CuDNNGRU)         (None, 10173, 185)        206460    
_________________________________________________________________
global_max_pooling1d (Global (None, 185)               0         
_________________________________________________________________
dropout (Dropout)            (None, 185)               0         
_________________________________________________________________
dense (Dense)                (None, 300)               55800     
_________________________________________________________________
dropout_1 (Dropout)          (None, 300)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 8)                 2

In [ ]:
x_test = test['content']
y_test = test['label_id']

print('Number of testing sentence: ', x_test.shape)
print('Number of testing label: ', y_test.shape)

Number of testing sentence:  (1644,)
Number of testing label:  (1644,)


In [ ]:
# Convert dataframes to numpy arrays
x_test = np.asarray(x_test)
y_test = np.asarray(y_test)

In [ ]:
# Applying preprocessing step to test data
test_docs = np.empty_like(x_test)
for index, document in enumerate(x_test):
  test_docs[index] = clean_data(document)

In [ ]:
# Embed testing sequences
encoded_docs = tokenizer.texts_to_sequences(test_docs)
x_test_padded = pad_sequences(encoded_docs, maxlen=max_length, padding='post')

In [ ]:
categorical_y_test = to_categorical(y_test, 8)

In [ ]:
# Evaluate model
loss, acc = model.evaluate(x_test_padded, categorical_y_test, verbose=1)
print('Test Accuracy: %f' % (acc*100))

52/52 [==============================] - 17s 273ms/step - loss: 0.2355 - accuracy: 0.9647
Test Accuracy: 96.472019


In [ ]:
y_pred = model.predict(x_test_padded)

In [ ]:
text = []
true_label = []
pred_label = []
angry = 0
happy = 0
neutral = 0
for i in range(len(y_pred)):
  text.append(x_test[i])
  true_label.append(y_test[i])
  print(x_test[i])
  print(y_test[i])
  print(np.argmax(y_pred[i]))

حسن جوهرچی بازیگر سینما تلویزیون ایران گفتگو خبرنگار حوزه سینما گروه فرهنگی باشگاه خبرنگاران جوان؛ خصوص علت کاری اش سال نظر بودجه کافی تولید بازیگران کار می شوند وضعیت بودجه بهبود رود تولیدات قدیم افزایش حرکت جدیدی اتفاق بیفتد دائم سرکار خصوص هوای سینما ایران بیان نظر سینما کم کم تعطیل است، آثار سینمایی مخاطب جذب نمی کند، سالن ها خالی فیلمسازان کاری فعالیت نمی کنند تاسف دارد، مسئولان سینمایی فکری موضوع خصوص تولید فیلم نامه سینما درخشان بازگردد ادامه بحث فیلم نامه موضوع تولید ساختار فیلم سینمای قدیم الایام مشکل صورتی اواخر دهه شصت اوایل دهه هفتاد آثار خوبی سینما می شد حرکت ادامه نکرد جوهرچی بیان دهه های مذکور فیلم های خوبی ناخدا خورشید، اجاره نشین ها ساختیم دلیل فیلم نامه نویسان فراغ بال بهتری کار می کردند ممیزی های گوناگون بحث فیلم نامه خودسانسوری  شروع موضوع تولید تلویزیون سال ها فیلم نامه ها خوبی تلویزیون مکه دست سینما بسته نیمه دهه هفتاد سینمای سیر نزولی می کند جوهرچی اظهار سیر نزولی نیمه دهه هفتاد فیلم های خوبی مادر، کمال الملک پیامد خوبی همراه موضوع مهم توجه فکری حالش بازیگر سینما

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

